# 03 Networks exploration

This notebook shows how networks were built and visualised based on the output from the topic model.

In [1]:
library(tidyverse)
library(tidytext)
library(topicmodels)

library(igraph)
library(sna)
library(ggraph)

library(wesanderson)
library(patchwork)
theme_set(theme_minimal())

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Dołączanie pakietu: ‘igraph’


Następujące obiekty zostały zakryte z ‘package:dplyr’:

    as_data_frame, groups, union


Następujące obiekty zostały zakryte z ‘package:purrr’:

    compose, simplify


Następujący obiekt został zakryty z ‘package:tidyr’:

    crossing


Następujący obiekt został zakryty z ‘package:tibble’:

    as_data_frame


Następujące obiekty zostały zakryte z ‘package:stats’:

    decompose, spectrum


Następujący obiekt został zakryty z ‘package:base’:

    union


Ładowanie wymaganego pakietu: statnet.common


Dołączanie pakietu: ‘statnet.common’


Następujące obiekty zostały zakryte z

In [13]:
# load model output
load("../data/lda75_output.Rda")

# load code for edges calculation
source("03_fn_calculate_edges.R")

# function to extract topic lables:
get_topic_labels = function(x, n_labels = 10) {
  
  #takes beta-matrix as an input
  top_terms = x %>%
    group_by(topic) %>%
    top_n(20, beta) %>%
    ungroup() %>%
    arrange(topic, -beta)
  
  topic_labels = top_terms %>%
    group_by(topic) %>%
    top_n(n_labels, beta) %>%
    summarise(label=paste(term, collapse=" ")) %>%
    mutate(label = paste(topic, label, sep="_"))
  
  return(topic_labels)
}

In [14]:
gamma <- gamma %>% 
    mutate(index = paste0(corpus, "_", id))

glimpse(gamma)

Rows: 1,076,700
Columns: 8
$ corpus     <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N",…
$ id         <chr> "1-1", "1-2", "1-3", "10-1", "10-2", "100-1", "100-2", "100…
$ year       <chr> "1785", "1785", "1785", "1803", "1803", "1810", "1810", "18…
$ first_line <chr> "ЕщеВкругСолнцевНеВращаться", "НеТвойЛиПламеньОбитать", "Он…
$ formula    <chr> "iamb_4", "iamb_4", "iamb_4", "iamb_4", "iamb_4", "iamb_6",…
$ topic      <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ gamma      <dbl> 0.019108280, 0.006211180, 0.006451613, 0.006211180, 0.00763…
$ index      <chr> "N_1-1", "N_1-2", "N_1-3", "N_10-1", "N_10-2", "N_100-1", "…


Extract topic labels

In [7]:
topic_labels <- get_topic_labels(beta, n_labels = 5)
head(topic_labels)

topic,label
<int>,<chr>
1,1_старик девица красный молодой старый
2,2_брат два жена муж другой
3,3_искать исчезать везде дух след
4,4_дева прекрасный краса чистый молодой
5,5_бог давать сила боже господь
6,6_слава великий герой отечество россия


Create edges: detect highly probable topics appeared in the same poems

In [16]:
edges_raw <- compute_edges(gamma,
                           gamma_thresh = 0.05,
                           time_slice = 5)

head(edges_raw)

edges_raw %>% 
  filter(corpus == "P") %>% 
  head

corpus,id,year,first_line,formula,topic,gamma,index,from,to,slice,edge_id,source,target
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
N,10256,1814,ОГореГореМы,iamb_free,1,0.06569343,N_10256,1,46,1810,1 46,1_старик девица красный молодой старый,46_стих писать читать перо поэт
N,10296-1,1848,ВдругНеХотетьЛиБратикВино,anapaest_регулярная:3+2#3,1,0.13548387,N_10296-1,1,15,1845,1 15,1_старик девица красный молодой старый,15_пить вино чаша пир гость
N,10296-1,1848,ВдругНеХотетьЛиБратикВино,anapaest_регулярная:3+2#3,1,0.13548387,N_10296-1,1,57,1845,1 57,1_старик девица красный молодой старый,57_другой хотеть иной становиться говорить
N,10296-1,1848,ВдругНеХотетьЛиБратикВино,anapaest_регулярная:3+2#3,1,0.13548387,N_10296-1,1,58,1845,1 58,1_старик девица красный молодой старый,58_день ночь тень час утро
N,10296-1,1848,ВдругНеХотетьЛиБратикВино,anapaest_регулярная:3+2#3,1,0.13548387,N_10296-1,1,59,1845,1 59,1_старик девица красный молодой старый,59_сказать становиться хорошо говорить приходить
N,10302,1850,СпешитьНаЗваныйПирПоУлицаПрегрязный,iamb_6,1,0.05660377,N_10302,1,3,1850,1 3,1_старик девица красный молодой старый,3_искать исчезать везде дух след


corpus,id,year,first_line,formula,topic,gamma,index,from,to,slice,edge_id,source,target
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
P,1007-1,1838,ПрийтиВзглядыватьВеликодушно,iamb_4,1,0.06285714,P_1007-1,1,13,1835,1 13,1_старик девица красный молодой старый,13_меч бой враг герой поле
P,1007-1,1838,ПрийтиВзглядыватьВеликодушно,iamb_4,1,0.06285714,P_1007-1,1,39,1835,1 39,1_старик девица красный молодой старый,39_ум труд свет наука глупец
P,1007-1,1838,ПрийтиВзглядыватьВеликодушно,iamb_4,1,0.06285714,P_1007-1,1,69,1835,1 69,1_старик девица красный молодой старый,69_царь народ русский царство князь
P,1007-1,1838,ПрийтиВзглядыватьВеликодушно,iamb_4,1,0.06285714,P_1007-1,1,73,1835,1 73,1_старик девица красный молодой старый,73_конь дорога поле бежать лететь
P,1029,1839,ЗимнийУтроВыходитьДевица,trochee_4,1,0.07692308,P_1029,1,32,1835,1 32,1_старик девица красный молодой старый,32_весна поле снег ветер зима
P,1030-2,1839,БытьДедушкаТыВТотВремя,trochee_5,1,0.12077295,P_1030-2,1,29,1835,1 29,1_старик девица красный молодой старый,29_слово сказать говорить знать никто


Create links for network

In [22]:
# select top meters
meter_counts <- gamma %>% 
  mutate(year_span = floor(as.numeric(year)/5)*5) %>% 
  rename("meter" = "formula") %>% 
  filter(!str_detect(meter, "other")) %>% 
  group_by(year_span, meter) %>% 
  count(sort = T) %>% 
  ungroup() %>% 
  filter(n > 10 & year_span != 1850)

  head(meter_counts)

year_span,meter,n
<dbl>,<chr>,<int>
1825,iamb_4,54975
1835,iamb_4,44775
1830,iamb_4,37050
1820,iamb_4,36450
1835,iamb_free,30525
1835,trochee_4,25425


In [33]:
top_meters <- meter_counts %>% 
  group_by(meter) %>%
  summarise(total = sum(n)) %>% 
  arrange(desc(total))

head(top_meters, 15)
top_meters$meter[1:8]

meter,total
<chr>,<int>
iamb_4,316425
iamb_free,183900
trochee_4,123300
iamb_6,117150
iamb_5,62100
iamb_3,25725
amphibrach_4,15150
iamb_регулярная:4+3,10425
amphibrach_3,6750


[1] "iamb_4"              "iamb_free"           "trochee_4"          
[4] "iamb_6"              "iamb_5"              "iamb_3"             
[7] "amphibrach_4"        "iamb_регулярная:4+3"

In [39]:
links <- edges_raw %>% 
  mutate(meter = formula) %>%
  # filter out only 8 most interesting meters
  filter(meter %in% top_meters$meter[1:8]) %>% 
  # group and count edges
  group_by(slice, meter, corpus) %>% 
  count(slice, meter, edge_id, source, target, sort = T) %>% 
  # filter non-frequent edges
  filter(n > 2) %>% 
  ungroup() 

# quick check
links %>% 
  filter(corpus == "P") %>% 
  head

slice,meter,corpus,edge_id,source,target,n
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1835,iamb_free,P,30 42,30_небо земля мир земной небесный,42_душа мечта поэт живой восторг,10
1835,iamb_4,P,24 72,24_вода луг тень ручей роща,72_гора лес скала дикий стоять,9
1835,iamb_4,P,30 74,30_небо земля мир земной небесный,74_луч солнце небо заря блистать,9
1835,iamb_4,P,8 74,8_звезда небо луна ночь луч,74_луч солнце небо заря блистать,9
1835,trochee_4,P,8 26,8_звезда небо луна ночь луч,26_волна море берег вода брег,8
1835,iamb_4,P,8 11,8_звезда небо луна ночь луч,11_святой молитва вера храм божий,7


Create edge and node lists

In [41]:
edgelist <- links %>% 
  select(source, target, n, meter, slice, corpus) %>% 
  mutate(width = n/10) %>% 
  filter(slice != 1850)

nodelist <- tibble(source = unique(c(links$target, links$source))) %>% 
  mutate(idn = as.numeric(str_replace(source, "^([0-9].*?)_.*", "\\1"))) 

 head(edgelist)
 head(nodelist) 
 nrow(nodelist)

source,target,n,meter,slice,corpus,width
<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<dbl>
41_лев волк зверь собака большой,57_другой хотеть иной становиться говорить,23,iamb_free,1780,N,2.3
36_знать говорить точно два очень,51_сидеть окно дверь дом стоять,20,trochee_4,1840,N,2.0
57_другой хотеть иной становиться говорить,59_сказать становиться хорошо говорить приходить,19,iamb_free,1780,N,1.9
20_богатый жить дом служить бедный,57_другой хотеть иной становиться говорить,16,iamb_free,1780,N,1.6
41_лев волк зверь собака большой,59_сказать становиться хорошо говорить приходить,16,iamb_free,1810,N,1.6
19_муза певец слава дар поэт,42_душа мечта поэт живой восторг,16,iamb_4,1825,N,1.6


source,idn
<chr>,<dbl>
57_другой хотеть иной становиться говорить,57
51_сидеть окно дверь дом стоять,51
59_сказать становиться хорошо говорить приходить,59
42_душа мечта поэт живой восторг,42
72_гора лес скала дикий стоять,72
62_знать хотеть ничто сказать бояться,62


[1] 75